# 露天采矿

## 目标和先决条件

采矿公司如何使用数学优化来确定应选择哪些开采地点以最大化开采矿石的毛利润？尝试这个建模示例来找出答案！

这个模型是H. Paul Williams所著《数学规划中的模型构建》第五版第269-270页和324-325页的示例14。

这是一个中级示例，我们假设您了解Python和Gurobi Python API，并且对构建数学优化模型有一定了解。

**下载代码库** <br /> 
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 问题描述

一家公司获得了在200英尺 × 200英尺的方形地块内进行露天采矿的许可。由于土壤的滑动角度限制，开挖边坡不可能超过45度。公司已经对不同地点不同深度的矿石价值进行了估算。考虑到滑动角度的限制，公司决定将问题视为矩形区块的开采问题。每个区块的水平尺寸为50英尺 × 50英尺，垂直尺寸为25英尺。如果区块被选择在彼此之上，那么只能开挖形成一个倒金字塔形状的区块组合。
下面的三维示意图显示了四个开采层级。我们用黑色数字标记了每个层级的每个区块，红色数字表示该层级下方的区块。例如，第2层的17号区块位于第1层的1、2、5和6号区块下方。
![pyramid](extractionPyramid.PNG)

已经估算了每个区块开采矿石的利润。目标是找到一个能够最大化总利润的矿石开采计划。

## 模型构建

### 集合和索引

$b,b2 \in \text{Blocks}=\{1,...,30 \}$: 区块集合

### 参数

$\text{profit}_{b} \in \mathbb{R}^+$: 开采区块 $b$ 的利润。

$(b,b2) \in Arcs = Blocks \times Blocks$: 此参数表示描述开采规则的串并联图中的弧。在此串并联图的邻接矩阵中，如果区块b2是区块b上方的四个区块之一，则弧$(b,b2)$的值为1，否则为0。例如，弧$(29,24)$表示24号区块是29号区块上方的四个区块之一。

### 决策变量

$\text{extract}_{b} \in \{0,1\}$: 如果选择开采区块 $b$，则此二元变量等于1，否则为0。

### 约束条件

**开采约束**: 如果开采某个区块，则其上方的四个区块也必须开采。

\begin{equation}
\text{extract}_{b2} \geq \text{extract}_{b} \quad \forall (b,b2) \in \text{Arcs}
\end{equation}

### 目标函数

**利润**: 最大化矿石开采的利润。

\begin{equation}
\text{最大化} \quad \sum_{b \in Blocks} \text{profit}_{b}*\text{extract}_{b}
\end{equation}

## Python实现

导入Gurobi Python模块。

In [ ]:
# %pip install gurobipy

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## 输入数据

定义模型和其他Python库所需的所有输入数据。

In [ ]:
# 创建字典来存储每个区块开采矿石的利润

blocks, profit = gp.multidict({
    ('1'): 0,
    ('2'): 0,
    ('3'): 0,
    ('4'): -1500,
    ('5'): 0,
    ('6'): 1000,
    ('7'): 0,
    ('8'): -1500,
    ('9'): -1000,
    ('10'): -1000,
    ('11'): -1500,
    ('12'): -2000,
    ('13'): -1500,
    ('14'): -1500,
    ('15'): -2000,
    ('16'): -2500,
    ('17'): 2000,
    ('18'): 2000,
    ('19'): -2000,
    ('20'): 0,
    ('21'): 0,
    ('22'): -4000,
    ('23'): -2000,
    ('24'): -2000,
    ('25'): -5000,
    ('26'): 16000,
    ('27'): 4000,
    ('28'): 2000,
    ('29'): 0,
    ('30'): 2000
})

# 创建串并联图邻接矩阵的字典

arcs, value = gp.multidict({
    ('30','26'): 1,
    ('30','27'): 1,
    ('30','28'): 1,
    ('30','29'): 1,
    ('29','21'): 1,
    ('29','22'): 1,
    ('29','24'): 1,
    ('29','25'): 1,
    ('28','20'): 1,
    ('28','21'): 1,
    ('28','23'): 1,
    ('28','24'): 1,
    ('27','18'): 1,
    ('27','19'): 1,
    ('27','21'): 1,
    ('27','22'): 1,
    ('26','17'): 1,
    ('26','18'): 1,
    ('26','20'): 1,
    ('26','21'): 1,
    ('25','11'): 1,
    ('25','12'): 1,
    ('25','15'): 1,
    ('25','16'): 1,
    ('24','10'): 1,
    ('24','11'): 1,
    ('24','14'): 1,
    ('24','15'): 1,
    ('23','9'): 1,
    ('23','10'): 1,
    ('23','13'): 1,
    ('23','14'): 1,
    ('22','7'): 1,
    ('22','8'): 1,
    ('22','11'): 1,
    ('22','12'): 1,
    ('21','6'): 1,
    ('21','7'): 1,
    ('21','10'): 1,
    ('21','11'): 1,
    ('20','5'): 1,
    ('20','6'): 1,
    ('20','9'): 1,
    ('20','10'): 1,
    ('19','3'): 1,
    ('19','4'): 1,
    ('19','7'): 1,
    ('19','8'): 1,
    ('18','2'): 1,
    ('18','3'): 1,
    ('18','6'): 1,
    ('18','7'): 1,
    ('17','1'): 1,
    ('17','2'): 1,
    ('17','5'): 1,
    ('17','6'): 1
})

## 模型部署

创建模型和变量。这些二元决策变量定义了从哪个区块开采矿石。

注意，约束条件的系数矩阵是完全单模的，因此决策变量可以定义在区间$[0,1]$内，问题可以作为线性规划问题求解。

In [ ]:
model = gp.Model('opencastMining')

# 定义从区块开采矿石的决策变量
extract = model.addVars(blocks, ub=1, vtype=GRB.CONTINUOUS, name="extract" )

Using license file c:\gurobi\gurobi.lic


以下约束确保如果开采某个区块，则其上方的四个区块也必须开采。

In [ ]:
# 开采约束条件

extractionConstrs = model.addConstrs( (extract[b] <= extract[b2]  for b,b2 in arcs), name='extractionConstrs')

我们的目标是最大化矿石开采的利润。

In [ ]:
# 目标函数

extractionProfit = gp.quicksum(profit[b]*extract[b] for b in blocks )

model.setObjective(extractionProfit, GRB.MAXIMIZE)

In [ ]:
# 验证模型公式

model.write('opencastMining.lp')

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 56 rows, 30 columns and 112 nonzeros
Model fingerprint: 0x83427493
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 22 rows and 12 columns
Presolve time: 0.01s
Presolved: 34 rows, 18 columns, 68 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3014500e+04   1.100300e+01   0.000000e+00      0s
       9    1.7500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds
Optimal objective  1.750000000e+04


## 分析

最优开采方案产生的总利润为 $\$17,500.00$。
下表显示了要开采的区块及其相关的利润或损失。

In [ ]:
# 输出报告

extraction_plan = pd.DataFrame(
    [(b, f"${profit[b]*round(extract[b].x):,.2f}") for b in blocks if (extract[b].x > 0.5)],
    columns = ["Block", "Profit/Loss"],
)
extraction_plan.index=[''] * len(extraction_plan)
extraction_plan

,Block,Profit/Loss
,1,$0.00
,2,$0.00
,3,$0.00
,5,$0.00
,6,"$1,000.00"
,7,$0.00
,9,"$-1,000.00"
,10,"$-1,000.00"
,11,"$-1,500.00"
,17,"$2,000.00"


## 参考文献

H. Paul Williams著《数学规划中的模型构建》第五版。

版权所有 © 2020 Gurobi Optimization, LLC